### Import Necessary Packages and Datasets

##### We start by importing pandas and numpy for data processing

In [1]:
import pandas as pd
import numpy as np

##### Here we're reading in all the csvs we used as datasets

In [25]:
votes = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/countypres_2000-2024.csv")
unemployment = pd.read_excel("https://raw.github.com/jamilditter/election_results/refs/heads/main/data/LAUS_Annual_Unemployment_By_State_2000to2024.xlsx", skiprows=3)
med_income = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/Median_Household_Income_By_State_FRED.csv")
age = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/State_by_Age.csv")
education = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/State_Education_by_Degree.csv")
population = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/Population%20by%20Age%20and%20Sex%20-%20US%2C%20States%2C%20Counties.csv")
bach_2024 = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/FRED_BachelorsEduAttainment2024.csv")
alaska_med_income = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/MEHOINUSAKA646N.csv")
rhode_is_med_income = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/MEHOINUSRIA672N_new.csv")
voter_participation = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/voter_participation.csv")

### Data Cleaning

##### After reading in all our csvs, we will start to prepare each dataset for a final merge. To do this, we're going to isolate the data we want, and sort them by year and state.

#### Votes

##### Here, we start by totalling the votes by state and year for the total number of votes, the total number of Democrat votes, and the total number of Republican votes. After this, we melt the dataframe so we can get a single column for each type of vote, indexed against the year and state from which those votes came from.

In [26]:
votes["totalvotes"] = votes.groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["dem"] = votes[votes["party"] == "DEMOCRAT"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["rep"] = votes[votes["party"] == "REPUBLICAN"].groupby(["state", "year"])["candidatevotes"].transform("sum")

totalvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["totalvotes"],
    var_name = "party",
    value_name = "total_votes"
).drop_duplicates().dropna().drop(columns=["party"])

demvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["dem"],
    var_name = "party",
    value_name = "dem_votes"
).drop_duplicates().dropna().drop(columns=["party"])

repvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["rep"],
    var_name = "party",
    value_name = "rep_votes"
).drop_duplicates().dropna().drop(columns=["party"])

#### Unemployment

##### Here, we start by creating a dictionary to rename the Series IDs to states so we can merge them later. 

In [27]:
rename_states = {
    'LAUST010000000000003': 'Alabama',
    'LAUST020000000000003': 'Alaska',
    'LAUST040000000000003': 'Arizona',
    'LAUST050000000000003': 'Arkansas',
    'LAUST060000000000003': 'California',
    'LAUST080000000000003': 'Colorado',
    'LAUST090000000000003': 'Connecticut',
    'LAUST100000000000003': 'Delaware',
    'LAUST110000000000003': 'District of Columbia',
    'LAUST120000000000003': 'Florida',
    'LAUST130000000000003': 'Georgia',
    'LAUST150000000000003': 'Hawaii',
    'LAUST160000000000003': 'Idaho',
    'LAUST170000000000003': 'Illinois',
    'LAUST180000000000003': 'Indiana',
    'LAUST190000000000003': 'Iowa',
    'LAUST200000000000003': 'Kansas',
    'LAUST210000000000003': 'Kentucky',
    'LAUST220000000000003': 'Louisiana',
    'LAUST230000000000003': 'Maine',
    'LAUST240000000000003': 'Maryland',
    'LAUST250000000000003': 'Massachusetts',
    'LAUST260000000000003': 'Michigan',
    'LAUST270000000000003': 'Minnesota',
    'LAUST280000000000003': 'Mississippi',
    'LAUST290000000000003': 'Missouri',
    'LAUST300000000000003': 'Montana',
    'LAUST310000000000003': 'Nebraska',
    'LAUST320000000000003': 'Nevada',
    'LAUST330000000000003': 'New Hampshire',
    'LAUST340000000000003': 'New Jersey',
    'LAUST350000000000003': 'New Mexico',
    'LAUST360000000000003': 'New York',
    'LAUST370000000000003': 'North Carolina',
    'LAUST380000000000003': 'North Dakota',
    'LAUST390000000000003': 'Ohio',
    'LAUST400000000000003': 'Oklahoma',
    'LAUST410000000000003': 'Oregon',
    'LAUST420000000000003': 'Pennsylvania',
    'LAUST440000000000003': 'Rhode Island',
    'LAUST450000000000003': 'South Carolina',
    'LAUST460000000000003': 'South Dakota',
    'LAUST470000000000003': 'Tennessee',
    'LAUST480000000000003': 'Texas',
    'LAUST490000000000003': 'Utah',
    'LAUST500000000000003': 'Vermont',
    'LAUST510000000000003': 'Virginia',
    'LAUST530000000000003': 'Washington',
    'LAUST540000000000003': 'West Virginia',
    'LAUST550000000000003': 'Wisconsin',
    'LAUST560000000000003': 'Wyoming'
}

unemployment["Series ID"] = unemployment["Series ID"].replace(rename_states)

##### We then remove "Annual\n" from the year columns so we can isolate the years.

In [28]:
unemployment_renamed = unemployment.rename(columns = lambda x: x[-4:]).rename(columns={"s ID": "state"})

##### Finally, we melt the dataframe to create an unemployment column indexed against state and year, matching our previous votes dataframe. We then filter the dataframe to only include the years we have voter information for (2000, 2004, 2008, 2012, 2016, 2020, 2024).

In [29]:
unemployment_melted = pd.melt(
    unemployment_renamed, id_vars = ["state"],
    value_vars = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"],
    var_name = "year",
    value_name = "unemployment_rate"
)

election_years = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"]
unemployment_melted = unemployment_melted[unemployment_melted["year"].isin(election_years)]

#### Median Income

##### Again we start by creating a dictionary to transform the state abbreviations into the full names of the states, so they match the other dataframes. We also convert the to just show the year, to keep it consistent between dataframes.

In [30]:
abbr_to_state = {
    "ALA": "Alabama",
    "AKA": "Alaska",
    "ARA": "Arkansas",
    "AZA": "Arizona",
    "CAA": "California",
    "COA": "Colorado",
    "CTA": "Connecticut",
    "DCA": "District of Columbia",
    "DEA": "Delaware",
    "FLA": "Florida",
    "GAA": "Georgia",
    "HIA": "Hawaii",
    "IDA": "Idaho",
    "ILA": "Illinois",
    "INA": "Indiana",
    "IAA": "Iowa",
    "KSA": "Kansas",
    "KYA": "Kentucky",
    "LAA": "Louisiana", 
    "MEA": "Maine",
    "MDA": "Maryland",
    "MAA": "Massachusetts",
    "MIA": "Michigan",
    "MNA": "Minnesota",
    "MSA": "Mississippi",
    "MOA": "Missouri",
    "MTA": "Montana",
    "NEA": "Nebraska",
    "NVA": "Nevada",
    "NHA": "New Hampshire",
    "NJA": "New Jersey",
    "NMA": "New Mexico",
    "NYA": "New York",
    "NCA": "North Carolina",
    "NDA": "North Dakota",
    "OHA": "Ohio",
    "OKA": "Oklahoma",
    "ORA": "Oregon",
    "PAA": "Pennsylvania",
    "RIA": "Rhode Island",
    "SCA": "South Carolina",
    "SDA": "South Dakota",
    "TNA": "Tennessee",
    "TXA": "Texas",
    "UTA": "Utah",
    "VAA": "Virginia",
    "VTA": "Vermont",
    "WAA": "Washington",
    "WIA": "Wisconsin",
    "WVA": "West Virginia",
    "WYA": "Wyoming"
}

med_income["observation_date"] = pd.to_datetime(med_income["observation_date"]).dt.year

c:\jamil\Temp\ipykernel_20964\4086705710.py:55: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  med_income["observation_date"] = pd.to_datetime(med_income["observation_date"]).dt.year


##### Our intial dataset was missing both Alaska and Rhode Island median incomes, so we downloaded this data separately, and have to merge it into the main dataframe. To do this, we need to convert the observation date columns into years, so they match up with the main median income dataframe.

In [31]:
alaska_med_income["observation_date"] = pd.to_datetime(alaska_med_income["observation_date"]).dt.year

In [32]:
rhode_is_med_income["observation_date"] = pd.to_datetime(rhode_is_med_income["observation_date"]).dt.year

##### After adjusting this, we can merge everything together, then rename the observation date column to year, so it matches the other dataframes we will need to merge it together with.

In [33]:
med_income_merged = pd.merge(alaska_med_income, med_income)
med_income_merged = pd.merge(rhode_is_med_income, med_income_merged)

In [34]:
med_income_renamed = med_income_merged.rename(columns={"observation_date": "year"})

##### Finally, we melt the dataframe, creating a single column for median income, indexed against state and year. And for the last step, we filter the dataframe to only include the election years.

In [35]:
med_income_melted = pd.melt(
    med_income_renamed, id_vars = ["year"],
    value_vars = ['MEHOINUSALA672N', 'MEHOINUSAKA646N', 'MEHOINUSARA672N', 'MEHOINUSAZA672N', 'MEHOINUSCAA672N', 
                  'MEHOINUSCOA672N', 'MEHOINUSCTA672N', 'MEHOINUSDCA672N', 'MEHOINUSDEA672N', 'MEHOINUSFLA672N', 
                  'MEHOINUSGAA672N', 'MEHOINUSHIA672N', 'MEHOINUSIAA672N', 'MEHOINUSIDA672N', 'MEHOINUSILA672N', 
                  'MEHOINUSINA672N', 'MEHOINUSKSA672N', 'MEHOINUSKYA672N', 'MEHOINUSLAA672N', 'MEHOINUSMAA672N', 
                  'MEHOINUSMDA672N', 'MEHOINUSMEA672N', 'MEHOINUSMIA672N', 'MEHOINUSMNA672N', 'MEHOINUSMOA672N', 
                  'MEHOINUSMSA672N', 'MEHOINUSMTA672N', 'MEHOINUSNEA672N', 'MEHOINUSNCA672N', 'MEHOINUSNDA672N', 
                  'MEHOINUSNHA672N', 'MEHOINUSNJA672N', 'MEHOINUSNMA672N', 'MEHOINUSNVA672N', 'MEHOINUSNYA672N', 
                  'MEHOINUSOHA672N', 'MEHOINUSOKA672N', 'MEHOINUSORA672N', 'MEHOINUSPAA672N', 'MEHOINUSSCA672N', 
                  'MEHOINUSSDA672N', 'MEHOINUSTNA672N', 'MEHOINUSTXA672N', 'MEHOINUSUTA672N', 'MEHOINUSVAA672N', 
                  'MEHOINUSVTA672N', 'MEHOINUSWAA672N', 'MEHOINUSWIA672N', 'MEHOINUSWVA672N', 'MEHOINUSWYA672N',
                  'MEHOINUSRIA672N'],
    var_name = "state",
    value_name = "median_income"
)

med_income_melted["state"] = med_income_melted["state"].str[8:11].map(abbr_to_state)
med_income_melted["year"] = med_income_melted["year"].astype(str)

med_income_melted = med_income_melted[med_income_melted["year"].isin(election_years)]

#### Median Age

##### We create a dictionary to convert the fips codes to state names.

In [36]:
fips_to_states = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming"
}

age["STATEFIP"] = age["STATEFIP"].replace(fips_to_states)

##### We then filter out everything but median age, state, and year, with median age indexed against state and year.

In [37]:
age = age.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

med_age = age.loc[:, ["state", "year", "med_age"]]

#### Bachelor's Attainment

##### We start by converting the fips codes to state names, and then renaming the columns to match our column naming scheme for the other dataframes.

In [38]:
education["STATEFIP"] = education["STATEFIP"].replace(fips_to_states)
education = education.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

##### We calculate the percentage of bachelor attainment by dividing the number of people with a bachelor's education or higher by the total population. As we are missing 2024 data for bachelor attainment, we had to import a dataset for just that year, which we prepare for merging by assigning it the year 2024 as a column.

In [39]:
education["bach_per"] = education["bach_pop"] / education["total_pop"]

bach_2024["year"] = 2024
bach_2024 = bach_2024.rename(columns = {"State": "state"})

##### Because the 2024 bachelor attainment is not represented as a percentage, we have to divide it by 100 so it matches our other bachelor attainment data.

In [40]:
bach_2024["2024 Bachelors or Higher"] = bach_2024["2024 Bachelors or Higher"]/100

##### We merge education dataframe with the 2024 bachelor's attainment dataframe, and then create a new dataframe filtering out all the variables except for bachelor attainment, state and year.

In [41]:
education_merge = pd.merge(education, bach_2024, on=["state", "year"], how="outer")
education_merge.loc[education_merge["year"] == 2024, "bach_per"] = education_merge.loc[education_merge["year"] == 2024, "2024 Bachelors or Higher"]
education_merge = education_merge.drop(columns = ["2024 Bachelors or Higher"])

In [42]:
edu_bach = education_merge.loc[:, ["state", "year", "bach_per"]]

#### Population

##### Here, we rename some columns to match the naming scheme of our other datasets, create a new variable representing the total voting age population, and then filtering our dataframe to only include voting age population, population aged 18-24, population aged 25-44, population aged 45-64, population aged 65+, state, and year. We then remove the national level data (USA as a whole), and filter the dataset to only include election years.

In [43]:
population_renamed = population.rename(columns = {"Total Population": "total_pop", "Year": "year", "Description": "state"})
population_renamed["voting_age_pop"] = population_renamed["Population 18-54"] + population_renamed["Population 55+"]
population_renamed = population_renamed.loc[:, ["state", "year", "voting_age_pop", "Population 18-24", "Population 25-44", "Population 45-64", "Population 65+"]]
population_renamed = population_renamed.loc[population_renamed["state"] != "U.S."]
population_renamed["year"] = population_renamed["year"].astype(str)
population_renamed = population_renamed[population_renamed["year"].isin(election_years)]

#### Voter Participation

##### Here, we melt voter participation to create a dataframe with voter participation indexed against state and year.

In [44]:
voter_participation_melt = pd.melt(voter_participation,
                                   id_vars="State",
                                   value_vars=["2000", "2004", "2008", "2012", "2016", "2020", "2024"],
                                    var_name = "year",
                                    value_name="voter_participation").iloc[:-6].rename(columns = {"State": "state"})

### Final Cleaning Steps

##### Here, we're merging all the different dataframes we've made together. We start by standardizing the column inputs for the charts, by converting all the year columns to integers, and capitalizing all the state columns. We then left join all the dataframes together on state and year. After creating a merged dataframe, we calculate the democratic voter percentage and Republican voter percentage by dividing the total number of the respective votes by the total votes for each state and year. Finally, we create a dummy variable that is True when the Democrats won a state, and False if the Republicans won instead.

In [45]:
unemployment_melted["year"] = unemployment_melted["year"].astype(int)
unemployment_melted["state"] = unemployment_melted["state"].str.upper()
med_income_melted["year"] = med_income_melted["year"].astype(int)
med_income_melted["state"] = med_income_melted["state"].str.upper()
med_age["state"] = med_age["state"].str.upper()
edu_bach["state"] = edu_bach["state"].str.upper()
population_renamed["state"] = population_renamed["state"].str.upper()
population_renamed["year"] = population_renamed["year"].astype(int)
voter_participation_melt["year"] = voter_participation_melt["year"].astype(int)
voter_participation_melt["state"] = voter_participation_melt["state"].str.upper()

merge1 = pd.merge(totalvotes_melted, demvotes_melted, on=["state", "year"], how="left")
merge2 = pd.merge(merge1, repvotes_melted, on=["state", "year"], how="left")
merge3 = pd.merge(merge2, unemployment_melted, on=["state", "year"], how="left")
merge4 = pd.merge(merge3, med_income_melted, on=["state", "year"], how="left")
merge5 = pd.merge(merge4, med_age, on=["state", "year"], how="left")
merge6 = pd.merge(merge5, edu_bach, on=["state", "year"], how="left")
merge7 = pd.merge(merge6, voter_participation_melt, on=["state", "year"], how="left")
merged_data = pd.merge(merge7, population_renamed, on=["state", "year"], how="left")

merged_data["dem_votes_per"] = merged_data["dem_votes"] / merged_data["total_votes"]
merged_data["rep_votes_per"] = merged_data["rep_votes"] / merged_data["total_votes"]

merged_data["dem_win"] = merged_data["dem_votes_per"] > merged_data["rep_votes_per"]

#### Exporting Cleaned CSV

##### The final step is to create an export path for the cleaned csv, and export it.

In [46]:
import os

save_path = "/Users/jamil/Coding/election_results/data/Voting_Breakdown_By_Socioeconomic_and_Demographic_Factors.csv"

os.makedirs(os.path.dirname(save_path), exist_ok=True)

merged_data.to_csv(save_path, index=False)